In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn import preprocessing

Après avoir rajouter les statistiques concernant les ratings et les tournois on souhaite intégrer des stats concernant le taux de succés des adversaires sur le type de terrain donné.

### Apprentissage

On reprend la table de données créée précédement.

In [2]:
df_match= pd.read_csv('train_tours_stats.csv')
df_match.head()

,ID1_G,ID2_G,ID_T_G,RANK_T,ID_R_G,ID_C_T,POINT_P1,POS_P1,POINT_P2,POS_P2,diff_rating,diff_wins_for_tour,diff_wins_for_tour_ratio,winner
0,935,838,8889,2,4,2,745.642857,74.523810,613.333333,90.742857,-16.219048,7.0,0.042156,1
1,21497,20655,8886,6,4,1,101.333333,452.368421,161.241176,350.505882,101.862539,1.0,-0.200000,2
2,11373,1264,8957,1,5,3,510.990476,114.242857,277.290476,220.085714,-105.842857,12.0,0.030094,2
3,15323,23433,8891,2,9,3,0.000000,1000.000000,0.000000,1000.000000,0.000000,28.0,-0.605263,1
4,6208,1049,8898,2,1,1,193.662500,289.425000,19.000000,848.666667,-559.241667,-5.0,-0.389655,1


In [3]:
df_stats_court = pd.DataFrame()
df_stats_court['j1_win']= df_match[df_match['winner'] == 1].groupby(['ID1_G', 'ID_C_T'])['winner'].count()
df_stats_court['j1_loose']= df_match[df_match['winner'] == 2].groupby(['ID1_G', 'ID_C_T'])['winner'].count()
df_stats_court['j2_win']= df_match[df_match['winner'] == 1].groupby(['ID2_G', 'ID_C_T'])['winner'].count()
df_stats_court['j2_loose']= df_match[df_match['winner'] == 2].groupby(['ID2_G', 'ID_C_T'])['winner'].count()
df_stats_court=df_stats_court.fillna(0)
df_stats_court['j_win']= df_stats_court.j1_win+ df_stats_court.j2_win
df_stats_court['j_loose']= df_stats_court.j2_loose+ df_stats_court.j2_loose
df_stats_court['j_total']= df_stats_court.j_win+ df_stats_court.j_loose
df_stats_court['ratio_court']= df_stats_court.j_win / df_stats_court.j_total
df_stats_court = df_stats_court.drop(['j1_win', 'j2_win', 'j1_loose', 'j2_loose','j_loose', 'j_total'], 1)
df_stats_court = df_stats_court.reset_index()
df_stats_court.head(10)

,ID1_G,ID_C_T,j_win,ratio_court
0,7,1,29.0,0.397260
1,7,2,8.0,0.800000
2,7,3,8.0,0.800000
3,7,5,18.0,0.409091
4,10,2,7.0,1.000000
5,12,1,5.0,0.714286
6,12,2,3.0,0.600000
7,12,3,6.0,0.600000
8,19,1,68.0,0.326923
9,19,2,33.0,0.407407


In [4]:
df_join_stats_court= df_match.merge(df_stats_court, on=['ID1_G', 'ID_C_T'], how='left')
df_join_stats_court.rename(columns={'j_win': 'j1_win_oncourt'}, inplace=True)
df_join_stats_court.rename(columns={'ratio_court': 'j1_win_oncourt_ratio'}, inplace=True)
df_join_stats_court= df_join_stats_court.merge(df_stats_court, left_on=['ID2_G', 'ID_C_T'], right_on=['ID1_G', 'ID_C_T'], how='left')
df_join_stats_court.rename(columns={'j_win': 'j2_win_oncourt'}, inplace=True)
df_join_stats_court.rename(columns={'ratio_court': 'j2_win_oncourt_ratio'}, inplace=True)
df_join_stats_court = df_join_stats_court.drop('ID1_G_y', 1)
df_join_stats_court.rename(columns={'ID1_G_x': 'ID1_G'}, inplace=True)
df_join_stats_court=df_join_stats_court.fillna(0)
df_join_stats_court['diff_wins_for_court']= df_join_stats_court.j1_win_oncourt - df_join_stats_court.j2_win_oncourt
df_join_stats_court['diff_wins_for_court_ratio']= df_join_stats_court.j1_win_oncourt_ratio - df_join_stats_court.j2_win_oncourt_ratio
df_join_stats_court=df_join_stats_court.drop(['j1_win_oncourt', 'j2_win_oncourt','j1_win_oncourt_ratio' , 'j2_win_oncourt_ratio'] , 1)
df_train_court_stats = df_join_stats_court[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_C_T', 'ID_R_G', 'POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio', 'diff_wins_for_court', 'diff_wins_for_court_ratio', 'winner']]
# df_train_tours_stats = df_join_stats_tours[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_R_G', 'ID_C_T', 'POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio', 'winner']]
df_train_court_stats.to_csv('train_court_stats.csv', sep=',', index=False)
df_train_court_stats.head(10)

,ID1_G,ID2_G,ID_T_G,RANK_T,ID_C_T,ID_R_G,POINT_P1,POS_P1,POINT_P2,POS_P2,diff_rating,diff_wins_for_tour,diff_wins_for_tour_ratio,diff_wins_for_court,diff_wins_for_court_ratio,winner
0,935,838,8889,2,2,4,745.642857,74.523810,613.333333,90.742857,-16.219048,7.0,0.042156,-5.0,0.044029,1
1,21497,20655,8886,6,1,4,101.333333,452.368421,161.241176,350.505882,101.862539,1.0,-0.200000,-10.0,-0.231884,2
2,11373,1264,8957,1,3,5,510.990476,114.242857,277.290476,220.085714,-105.842857,12.0,0.030094,20.0,0.155112,2
3,15323,23433,8891,2,3,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,28.0,-0.605263,9.0,-0.645161,1
4,6208,1049,8898,2,1,1,193.662500,289.425000,19.000000,848.666667,-559.241667,-5.0,-0.389655,6.0,0.145749,1
5,22609,25447,8899,3,1,4,0.000000,1000.000000,0.000000,1000.000000,0.000000,-2.0,-0.500000,1.0,-0.071429,2
6,3468,6431,9091,1,2,4,0.000000,1000.000000,454.904762,155.333333,844.666667,-56.0,-0.518519,-45.0,-0.483871,2
7,22347,25700,8903,3,2,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,13.0,-0.045455,6.0,-0.166667,1
8,17020,4178,9085,1,1,5,138.546053,341.342105,271.542484,276.352941,64.989164,-2.0,0.007590,-21.0,-0.148432,2
9,25813,21659,8906,2,2,9,0.000000,1000.000000,0.000000,1000.000000,0.000000,-1.0,0.500000,-1.0,0.500000,2


In [ ]:
X= df_train_court_stats.values

In [ ]:
clf = RandomForestClassifier()
clf.fit(X[:60000,:15], X[:60000,15])  

In [ ]:
print clf.score(X[:60000,:15], X[:60000,15])
print clf.score(X[60000:,:15], X[60000:,15])

In [ ]:
# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250, random_state=0)

forest.fit(X[:,:15], X[:,15])
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1] -1):
    print("%d. feature %d (%f)" % (f + 1, indices[f]+1, importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]-1), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]-1), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
X_normalized = preprocessing.normalize(X, norm='l2')
print X_normalized

In [ ]:
clf_svm = svm.SVC()
clf_svm.fit(X[:60000, :15], X[:60000, 15])

In [ ]:
clf_svmvmsvmsvm.

In [ ]:
test_public= pd.read_csv('test_tours_stats_.csv')
test_public.head()

In [ ]:
df_join_stats_court= test_public.merge(df_stats_court, on=['ID1_G', 'ID_C_T'], how='left')
df_join_stats_court.rename(columns={'j_win': 'j1_win_oncourt'}, inplace=True)
df_join_stats_court.rename(columns={'ratio_court': 'j1_win_oncourt_ratio'}, inplace=True)
df_join_stats_court= df_join_stats_court.merge(df_stats_court, left_on=['ID2_G', 'ID_C_T'], right_on=['ID1_G', 'ID_C_T'], how='left')
df_join_stats_court.rename(columns={'j_win': 'j2_win_oncourt'}, inplace=True)
df_join_stats_court.rename(columns={'ratio_court': 'j2_win_oncourt_ratio'}, inplace=True)
df_join_stats_court = df_join_stats_court.drop('ID1_G_y', 1)
df_join_stats_court.rename(columns={'ID1_G_x': 'ID1_G'}, inplace=True)
df_join_stats_court=df_join_stats_court.fillna(0)
df_join_stats_court['diff_wins_for_court']= df_join_stats_court.j1_win_oncourt - df_join_stats_court.j2_win_oncourt
df_join_stats_court['diff_wins_for_court_ratio']= df_join_stats_court.j1_win_oncourt_ratio - df_join_stats_court.j2_win_oncourt_ratio
df_join_stats_court=df_join_stats_court.drop(['j1_win_oncourt', 'j2_win_oncourt','j1_win_oncourt_ratio' , 'j2_win_oncourt_ratio'] , 1)
df_test_court_stats = df_join_stats_court[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_C_T', 'ID_R_G','POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio', 'diff_wins_for_court', 'diff_wins_for_court_ratio']]
df_test_court_stats.to_csv('test_court_stats.csv', sep=',', index=False)
df_test_court_stats.head(10)

In [ ]:
X_test= df_test_court_stats.values

In [ ]:
pred  = clf.predict(X_test)
print pred

In [ ]:
df_leaderboard_court_stats = df_test_court_stats[['ID1_G', 'ID2_G', 'ID_T_G', 'RANK_T', 'ID_C_T', 'ID_R_G','POINT_P1', 'POS_P1', 'POINT_P2','POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio', 'diff_wins_for_court', 'diff_wins_for_court_ratio']]
df_leaderboard_court_stats['predictions'] = pred
df_leaderboard_court_stats = df_leaderboard_court_stats.drop(['RANK_T', 'ID_C_T', 'POINT_P1', 'POINT_P2', 'POS_P1', 'POS_P2', 'diff_rating', 'diff_wins_for_tour', 'diff_wins_for_tour_ratio', 'diff_wins_for_court', 'diff_wins_for_court_ratio'], 1)
df_leaderboard_court_stats.head(10)

In [ ]:
df_leaderboard_court_stats.to_csv('leaderboard_court_stats.csv', sep=',', index=False, header= False)

In [ ]:
df_leaderboard_court_stats.shape